# IRS Data

In [1]:
import duckdb

duck = duckdb.connect()

In [3]:
duck.execute("create or replace table raw_fl_cd AS SELECT * FROM read_xlsx('https://www.irs.gov/pub/irs-soi/22incdfl.xlsx', range='A4:FG', all_varchar=true, header=true);")

duck.execute("show table raw_fl_cd").df()

,column_name,column_type,null,key,default,extra
0,Congressional\ndistrict [1],VARCHAR,YES,None,None,None
1,Size of adjusted gross income by congressional...,VARCHAR,YES,None,None,None
2,Number of returns [2],VARCHAR,YES,None,None,None
3,Number of single returns,VARCHAR,YES,None,None,None
4,Number of joint returns,VARCHAR,YES,None,None,None
...,...,...,...,...,...,...
158,_72,VARCHAR,YES,None,None,None
159,Overpayments refunded [16],VARCHAR,YES,None,None,None
160,_73,VARCHAR,YES,None,None,None
161,Credited to next year's estimated tax,VARCHAR,YES,None,None,None


In [4]:
duck.sql("""
WITH brackets AS (
    PIVOT (
        SELECT 
            "Congressional\ndistrict [1]" AS district,
            "Size of adjusted gross income by congressional district" AS income_bracket,
            -- Remove commas and cast to numeric to allow math operations
            CAST(REPLACE("Number of returns [2]", ',', '') AS DOUBLE) AS return_count
        FROM raw_fl_cd
        WHERE try("Congressional\ndistrict [1]"::int) BETWEEN 1 AND 999
    )
    ON income_bracket
    USING FIRST(return_count)
    GROUP BY district
)
SELECT 
    *, 
    ("$500,000 or more" / "$100,000 under $200,000") AS richness_ratio
FROM brackets
ORDER BY richness_ratio DESC;
""").df()

,district,"$1 under $10,000","$10,000 under $25,000","$100,000 under $200,000","$200,000 under $500,000","$25,000 under $50,000","$50,000 under $75,000","$500,000 or more","$75,000 under $100,000",Under $1,richness_ratio
0,27,44560.0,74960.0,49020.0,33050.0,79920.0,46070.0,19370.0,28010.0,10030.0,0.395145
1,22,41050.0,69820.0,63050.0,35890.0,82940.0,54840.0,16100.0,34740.0,10310.0,0.255353
2,19,37670.0,69620.0,72010.0,36980.0,95890.0,61040.0,18150.0,39560.0,8640.0,0.252048
3,21,42550.0,76960.0,60730.0,29750.0,88930.0,55210.0,12600.0,34050.0,9480.0,0.207476
4,18,39430.0,70900.0,70430.0,32110.0,90980.0,59140.0,13040.0,39460.0,9150.0,0.185148
5,23,40700.0,70810.0,61610.0,30070.0,81350.0,52480.0,10540.0,33890.0,9760.0,0.171076
6,14,39530.0,76950.0,51860.0,26230.0,94460.0,53230.0,8820.0,31200.0,6200.0,0.170073
7,24,47150.0,111000.0,27120.0,9730.0,109940.0,48750.0,3660.0,23040.0,7200.0,0.134956
8,16,41700.0,74150.0,82250.0,35000.0,102670.0,66580.0,10850.0,44690.0,8830.0,0.131915
9,13,35300.0,64230.0,52600.0,20560.0,91000.0,56750.0,6860.0,34040.0,7600.0,0.130418
